In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import BeerBenchmark

benchmark = BeerBenchmark(data_dir_path="../data/")
benchmark

01:25:40 INFO:Extracting Beer...
01:25:40 INFO:Reading Beer row_dict...
01:25:40 INFO:Reading Beer train.csv...
01:25:40 INFO:Reading Beer valid.csv...
01:25:40 INFO:Reading Beer test.csv...


<BeerBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Beer/beer_exp_data.zip

## Preprocess

In [5]:
attr_list = ['Beer_Name', 'Brew_Factory_Name', 'Style', 'ABV']

In [6]:
from tqdm.auto import tqdm
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

  0%|          | 0/7345 [00:00<?, ?it/s]

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_config_dict = {
    'Beer_Name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'semantic_Beer_Name': {
        'source_attr': 'Beer_Name',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'Brew_Factory_Name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'semantic_Brew_Factory_Name': {
        'source_attr': 'Brew_Factory_Name',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'Style': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'semantic_Style': {
        'source_attr': 'Style',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'ABV': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
}

In [10]:
from entity_embed import AttrConfigDictParser

row_numericalizer = AttrConfigDictParser.from_dict(attr_config_dict, row_list=benchmark.row_dict.values())
row_numericalizer.attr_config_dict

01:25:40 INFO:For attr=Beer_Name, computing actual max_str_len
01:25:40 INFO:For attr=Beer_Name, using actual_max_str_len=18
01:25:40 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
01:25:44 INFO:For attr=Brew_Factory_Name, computing actual max_str_len
01:25:44 INFO:For attr=Brew_Factory_Name, using actual_max_str_len=20
01:25:44 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
01:25:47 INFO:For attr=Style, computing actual max_str_len
01:25:47 INFO:actual_max_str_len=13 must be even to enable NN pooling. Updating to 14
01:25:47 INFO:For attr=Style, using actual_max_str_len=14
01:25:47 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
01:25:50 INFO:For attr=ABV, computing actual max_str_len
01:25:50 INFO:actual_max_str_len=7 must be even to enable NN pooling. Updating to 8
01:25:50 INFO:For attr=ABV, using actual_max_str_len=8


{'Beer_Name': AttrConfig(source_attr='Beer_Name', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=18, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True),
 'semantic_Beer_Name': AttrConfig(source_attr='Beer_Name', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '

In [11]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=16,
    eval_batch_size=128,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import Matcher

model = Matcher(
    row_numericalizer=row_numericalizer,
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStoppingMinEpochs(
   monitor='valid_f1_at_0.5',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_save_dir = '../tb_logs'
tb_name = f'matcher-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_save_dir, name=tb_name),
    reload_dataloaders_every_epoch=True
)

GPU available: True, used: True
01:25:50 INFO:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
01:25:50 INFO:TPU available: None, using: 0 TPU cores


In [14]:
trainer.fit(model, datamodule)

01:25:50 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type              | Params
--------------------------------------------------
0 | matcher_net | MatcherNet        | 10.5 M
1 | loss_fn     | BCEWithLogitsLoss | 0     
--------------------------------------------------
7.4 M     Trainable params
3.1 M     Non-trainable params
10.5 M    Total params
41.936    Total estimated model params size (MB)
01:25:53 INFO:
  | Name        | Type              | Params
--------------------------------------------------
0 | matcher_net | MatcherNet        | 10.5 M
1 | loss_fn     | BCEWithLogitsLoss | 0     
--------------------------------------------------
7.4 M     Trainable params
3.1 M     Non-trainable params
10.5 M    Total params
41.936    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

## Testing

In [15]:
trainer.test(ckpt_path='best', verbose=False)

01:27:12 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{'test_f1_at_0.3': 0.6842105388641357,
  'test_f1_at_0.4': 0.7222222089767456,
  'test_f1_at_0.5': 0.75,
  'test_precision_at_0.3': 0.5416666865348816,
  'test_precision_at_0.4': 0.5909090638160706,
  'test_precision_at_0.5': 0.6666666865348816,
  'test_recall_at_0.3': 0.9285714030265808,
  'test_recall_at_0.4': 0.9285714030265808,
  'test_recall_at_0.5': 0.8571428656578064}]